In [205]:
import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import max_error, mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [2]:
TRAIN_FILE = "Train.csv"
DEFAULT_CATEGORICAL_VALUE = "NAN"

In [3]:
df = pd.read_csv(TRAIN_FILE)

In [4]:
df.head()

,name,summary,description,neighborhood_overview,transit,host_since,host_location,host_about,host_response_rate,host_acceptance_rate,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,reviews_per_month
0,Guest Quarters on Capitol Hill,The Guest Quarters on Capitol Hill is a garde...,The Guest Quarters on Capitol Hill is a garde...,the convenience and charm of being on Capitol ...,The Red Line and the Blue Line Metros are with...,2013-08-14,"Washington, District of Columbia, United States",We are a reservation service for many DC area ...,98%,52%,...,10.0,10.0,10.0,10.0,10.0,8.0,f,strict,41,0.18
1,Private oasis. Top floor w/ 2 decks,"Top two floors, two bathrooms and two decks. M...","Top two floors, two bathrooms and two decks. M...",Adams Morgan is the most vibrant part of DC wh...,There is so many transportation options. Acros...,2012-01-23,"Washington, District of Columbia, United States","I have lived in DC for that past five years, b...",100%,100%,...,9.0,9.0,10.0,9.0,10.0,9.0,f,strict,1,0.41
2,People's Place -Convention Center,NaN,This 2 bedroom apartment is on the ground floo...,NaN,NaN,2009-10-31,"Berkeley, California, United States",A professional photo editor and photographer l...,97%,100%,...,8.0,8.0,9.0,9.0,9.0,9.0,f,strict,3,1.09
3,M St Apt C,"Inspired by MidEast styles. 1 full bed, 1 air ...","Inspired by MidEast styles. 1 full bed, 1 air ...",I love this neighborhood! Mt. Vernon is the ve...,The Mt. Vernon Sq metro is just 2 blocks away ...,2013-04-23,"San Francisco, California, United States","I am from Oakland, California and I like to li...",92%,100%,...,9.0,8.0,8.0,8.0,9.0,9.0,f,flexible,4,4.34
4,Columbia Heights Rm w/Private Bath,Enjoy our dressed up guest room in the heart o...,Enjoy our dressed up guest room in the heart o...,NaN,NaN,2012-07-09,"Washington, District of Columbia, United States",I'm a tree-hugger at heart. My wife is an arti...,100%,75%,...,9.0,9.0,10.0,10.0,8.0,9.0,f,moderate,3,0.09


## В нашем датасете присутствуют разные типы данных: числовые, категориальные, текстовые, datetime объекты. Чтобы не выявлять эти типы вручную, автоматизируем этот процесс

In [5]:
# с числовыми работать проще всего, поэтому выделим и проанализируем нечисловые 
not_numerical = list()
numerical = list()
for col in df.columns:
    try:
        while True:
            value = df[col][np.random.randint(0, 3012)]
            if value is np.nan:
                continue
            else:
                break
        # числовое значение в этом месте не вызовет ошибки
        v = float(value)
        numerical.append(col)
    except ValueError:
        not_numerical.append(col)

In [6]:
not_numerical

['name',
 'summary',
 'description',
 'neighborhood_overview',
 'transit',
 'host_since',
 'host_location',
 'host_about',
 'host_response_rate',
 'host_acceptance_rate',
 'host_neighbourhood',
 'host_verifications',
 'neighbourhood_cleansed',
 'city',
 'state',
 'market',
 'smart_location',
 'country_code',
 'country',
 'property_type',
 'room_type',
 'bed_type',
 'amenities',
 'price',
 'cleaning_fee',
 'instant_bookable',
 'cancellation_policy']

In [7]:
numerical

['zipcode',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'guests_included',
 'minimum_nights',
 'maximum_nights',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'calculated_host_listings_count',
 'reviews_per_month']



**В нечисловых данных:**

**name - тексовая переменная - скорее всего название отеля, у каждого уникальное, вряд ли несёт полезную информацию**

**summary - текстовая переменная - описание, может содержать ключевые слова, влияющие на конечную цену**

**description - текстовая переменная - то же самое, что и summary, можно просто отбросить, оставить один из них**

**neighborhood_overview - текстовая переменная - описание того, что по соседству, скорее всего важный параметр**

**transit - текстовая переменная - описание того, насколько удобно расположение относительно транспорта, важно**

**host_since - datetime объект - дата**

**host_location - текстовая переменная - расположение, может влиять на цену, так как стоимость жилья в США сильно зависит от штата**

**host_about - текстовая переменная - как хозяин владельцы описывают себя**

**host_response_rate - числовая переменная, нужно форматирование**

**host_acceptance_rate - числовая переменная, нужно форматирование**

**host_neighborhood - текстовая или категориальная - что находится по соседству**

**neighborhood_cleansed - текстовая**

**city - категориальная - город, важно**

**state - категориальная - штат, важно**

**market - категориальная - только одно значение 'D.C', можно отбросить**

**smart_location - локация**

**country_code - категориальная переменная - код страны, может влиять, но в датасете только значение 'US', так что это не даст информации**

**country - категориальная переменная - название страны, несёт ту же информацию, что и country_code, можно отбросить**

**property_type - категориальная переменная - тип жилья, скорее всего важно**

**room_type - категориальная переменная - тип комнаты, важно**

**bed_type - тип кровати, может быть важно**

**amenities - после форматирования можно сделать текстовой переменной - удобства, важно**

**price - числовая переменная после форматирования - целевая переменная, цена**

**cleaning_fee - числовая - оплата за уборку**

**instant_bookable - категориальная(бинарная) переменная - что-то связанное с заказом жилья**

**cancellation_policy - категориальная переменная - политика отмены**


In [8]:
# категориальные переменные
categorical = ["city", "state", "market", "smart_location", "country_code",
              "country", "property_type", "room_type", "bed_type", "instant_bookable", "cancellation_policy"]

# datetime объекты

datetimes = ["host_since"]

# текстовые переменные
text = ["name", "summary", "description", "neighborhood_overview", "transit", "host_location", "host_about", 
       "host_neighbourhood", "host_verifications", "amenities", "neighbourhood_cleansed"]

# переменные, отображающие процентное соотношение
percents = ["host_response_rate", "host_acceptance_rate"]

# переменные, обозначающие цену
prices = ["price", "cleaning_fee"]

In [9]:
# проверяем, всё ли мы разобрали
len(categorical) + len(text) + len(percents) + len(prices) + len(numerical) + len(datetimes)

45

In [11]:
# посмотрим, какие из категориальных переменных имеют одно значение
for col in categorical:
    print(col, df[col].unique(), "\n\n")

city ['Washington' 'Washington, D.C.' 'Silver Spring' 'Mount Rainier'
 'New York' 'Columbia Heights, middle of DC!' 'Washington ' 'Arlington'
 'Takoma Park' 'Dupont Circle' 'Annapolis' 'Hyattsville'
 'Capitol Hill, Washington' 'Adams Morgan'] 


state ['DC' 'MD' 'NY' 'VA' 'Washington DC'] 


market ['D.C.'] 


smart_location ['Washington, DC' 'Washington, D.C., DC' 'Silver Spring, MD'
 'Mount Rainier, MD' 'New York, NY' 'Columbia Heights, middle of DC!, DC'
 'Washington , DC' 'Arlington, VA' 'Takoma Park, MD' 'Dupont Circle, DC'
 'Annapolis, MD' 'Hyattsville, MD' 'Capitol Hill, Washington, DC'
 'Adams Morgan, Washington DC'] 


country_code ['US'] 


country ['United States'] 


property_type ['Apartment' 'House' 'Townhouse' 'Bed & Breakfast' 'Condominium' 'Cabin'
 'Other' 'Loft' 'Boat' 'Dorm' nan 'Bungalow'] 


room_type ['Entire home/apt' 'Private room' 'Shared room'] 


bed_type ['Real Bed' 'Pull-out Sofa' 'Futon' 'Couch' 'Airbed'] 


instant_bookable ['f' 't'] 


cancellation_polic

In [12]:
transformed_df = df.copy()

In [13]:
# отбросим переменные с одинаковым значением и name
transformed_df.drop(["name", "country", "country_code", "market"], inplace=True, axis=1)

In [14]:
# определим функции для извлечения чисел из цен и процентных соотношений
        
def get_price_helper(x):
    if x is not np.nan and '$' in x:
        return float(x[1:].replace(",", ""))
    else:
        return np.nan
    
    
def get_percents_helper(x):
    if x is not np.nan and '%' in x:
        return float(x[:-1])
    else:
        return np.nan
    
    
def price_transformer(column):
    arr = column.to_numpy()
    price = np.vectorize(get_price_helper)(arr)
    return price


def percents_transformer(column):
    arr = column.to_numpy()
    percents = np.vectorize(get_percents_helper)(arr)
    return percents

In [15]:
# определим функцию для перевода datetime объектов в timestamp

def datetime_helper(x, pattern):
    if x is not np.nan:
        return datetime.datetime.strptime(x, pattern).timestamp()
    return np.nan

def datetime_to_timestamp(column, pattern):
    arr = column.to_numpy()
    timestamp = np.vectorize(datetime_helper)(arr, pattern)
    return timestamp

In [16]:
# определим функции для векторизации текста

def tfidf_vectorizer(df, columns):
    # очистку текста от стоп-слов и лемматизацию оставляем на встроенный в класс TfidfVectorizer tokenizer
    out_features = None
    for column in columns:
        # здесь размер выходного вектора будет длиной в количество слов, нак оторых мы делали fit, эти вектора 
        # хранятся как sparse-матрицы для экономии места
        tfidf = TfidfVectorizer() 
        tfidf.fit(df[column])
        if out_features is None:
            out_features = tfidf.transform(df[column])
        else:
            out_features = hstack((out_features, tfidf.transform(df[column])))
    return out_features


def hashing_vectorizer(df, columns, n_features):
    hasher = HashingVectorizer(n_features=n_features)
    out_features = None
    for column in columns:
        if out_features is None:
            out_features = hasher.transform(df[column])
        else:
            out_features = hstack((out_features, hasher.transform(df[column])))
    return out_features

In [17]:
# функция для кодирования категориальных переменных

def fill_categorical_label(dataset, columns):
    for column_name in columns:
        label_encoder = LabelEncoder().fit(np.sort(dataset[column_name].unique()))
        dataset[column_name] = dataset[column_name].map(
            dict(zip(dataset[column_name].unique(), label_encoder.transform(dataset[column_name].unique()))))

In [19]:
# функции для заполнения отсутствующих значений
def numerical_fill(df, columns):
    for column in columns:
        # заполняем медианой 
        df[column].fillna(df[column].median(), inplace=True)
        
# функция для заполнения отсутствующих значений в категориальных переменных
def categorical_fill(df, columns):
    for column in columns:
        # заполняем значением по умолчанию, также можно будет попробовать заполнять самым частым значением
        df[column].fillna(df[column].mode(), inplace=True)


## Преобразуем цены и проценты в числовые переменные

In [20]:
transformed_df["price"] = price_transformer(transformed_df["price"])

In [21]:
transformed_df["cleaning_fee"] = price_transformer(transformed_df["cleaning_fee"])

In [22]:
transformed_df["host_response_rate"] = percents_transformer(transformed_df["host_response_rate"])

In [23]:
transformed_df["host_acceptance_rate"] = percents_transformer(transformed_df["host_acceptance_rate"])

In [32]:
# переведём datetime в timestamp
transformed_df["host_since"] = datetime_to_timestamp(transformed_df["host_since"], "%Y-%m-%d")

In [24]:
# мы сделали проценты и цены численными
numerical.extend(prices)
numerical.extend(percents)

In [26]:
# пока что отбросим поле zipcode, так как там есть и строки как с числами, так и с символами
transformed_df.drop("zipcode", axis=1, inplace=True)

In [28]:
numerical.remove("zipcode")

## На этом моменте я разделю датасет на 3: с числовыми переменными, с категориальными и с текстом

In [33]:
train_set = transformed_df.drop("price", axis=1)

In [36]:
y = transformed_df["price"]

In [37]:
numerical.remove("price")

In [38]:
numerical_set = train_set[numerical]

In [ ]:
# удалим из категориальных переменные с одинаковым значением
categorical.remove("country")
categorical.remove("country_code")

In [41]:
categorical.remove("market")

In [42]:
categorical

['city',
 'state',
 'smart_location',
 'property_type',
 'room_type',
 'bed_type',
 'instant_bookable',
 'cancellation_policy']

In [80]:
categorical_set = train_set[categorical]

In [45]:
# удалим поле name из-за неинформативности
text.remove("name")

In [46]:
text_set = train_set[text]

In [50]:
# Заполним пустые значения
numerical_fill(numerical_set, numerical)

/home/alexey/development/lib/python3.8/site-packages/pandas/core/series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/alexey/development/lib/python3.8/site-packages/pandas/core/series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/alexey/development/lib/python3.8/site-packages/pandas/core/series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super

In [51]:
numerical_set.isna().sum()

accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
guests_included                   0
minimum_nights                    0
maximum_nights                    0
number_of_reviews                 0
review_scores_rating              0
review_scores_accuracy            0
review_scores_cleanliness         0
review_scores_checkin             0
review_scores_communication       0
review_scores_location            0
review_scores_value               0
calculated_host_listings_count    0
reviews_per_month                 0
cleaning_fee                      0
host_response_rate                0
host_acceptance_rate              0
dtype: int64

In [81]:
categorical_set.fillna(categorical_set.mode(), inplace=True)

/home/alexey/development/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [86]:
categorical_set.isna().sum()

city                   0
state                  0
smart_location         0
property_type          1
room_type              0
bed_type               0
instant_bookable       0
cancellation_policy    0
dtype: int64

In [87]:
# почему-то в столбце property_type осталось одно пустое значение
categorical_set = pd.concat([categorical_set, y], axis=1)

In [89]:
categorical_set.dropna(inplace=True)


In [92]:
cat_y = categorical_set["price"]
categorical_set.drop("price", axis=1, inplace=True)

In [97]:
encoded_categorical = categorical_set.copy()
fill_categorical_label(encoded_categorical, categorical)

In [69]:
# в текстовых данных заполним пустой строкой 

In [70]:
text_set.fillna("", inplace=True)

/home/alexey/development/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [72]:
text_set.isna().sum()

summary                   0
description               0
neighborhood_overview     0
transit                   0
host_location             0
host_about                0
host_neighbourhood        0
host_verifications        0
amenities                 0
neighbourhood_cleansed    0
dtype: int64

# Проверка моделей

In [130]:
# функция для перекрёстной оценки

def kfold_test(x, y, model, folds=5):
    kfold = KFold(n_splits=folds)
    for train_index, test_index in kfold.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(x_train, y_train)
        y_train_pred = model.predict(x_train)
        y_test_pred = model.predict(x_test)
        print("train error", mean_absolute_error(y_train, y_train_pred))
        print("test error", mean_absolute_error(y_test, y_test_pred), "\n\n")
        
def kfold_test_text(x, y, model, folds=5):
    kfold = KFold(n_splits=folds)
    for train_index, test_index in kfold.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(x_train, y_train)
        y_train_pred = model.predict(x_train)
        y_test_pred = model.predict(x_test)
        print("train error", mean_absolute_error(y_train, y_train_pred))
        print("test error", mean_absolute_error(y_test, y_test_pred), "\n\n")
        

    
    

# Linear Regression

**Числовые переменные**

In [102]:
lin_reg = Pipeline(steps=[
    ("scaler", StandardScaler()), # масштабируем данные
    ("lin_reg", LinearRegression())
])

In [128]:
kfold_test(numerical_set, y, lin_reg)

train error 57.35581016688911
test error 51.031943117812474 


train error 52.03108481697439
test error 61.247344063198234 


train error 55.85709661504433
test error 93.50316521340066 


train error 54.12203003235305
test error 53.783627585154505 


train error 54.75768384363731
test error 58.048978321049184 




**Категориальные переменные**

In [129]:
kfold_test(encoded_categorical, cat_y, lin_reg)

train error 71.18316997207961
test error 60.588130793961234 


train error 65.30464982734895
test error 77.57669712461134 


train error 69.84098950349932
test error 66.41699895256201 


train error 67.37351225033107
test error 69.91796157370197 


train error 68.73110236643734
test error 69.16031118581559 




**Текстовые данные. Векторизуем какое-то из текстовых полей**

In [114]:
text_train = tfidf_vectorizer(text_set, ["description"])

In [116]:
text_train

<3013x9812 sparse matrix of type '<class 'numpy.float64'>'
	with 272584 stored elements in Compressed Sparse Row format>

In [133]:
# размерность - 9812, встроим в pipeline декомпозицию PCA
text_lin_reg = Pipeline(steps=[
    ("svd", TruncatedSVD(n_components=100)),
    ("scaler", StandardScaler()),
    ("lin_reg", LinearRegression())
])

In [134]:
kfold_test_text(text_train, y, text_lin_reg)

train error 68.75626724786794
test error 59.41750728988262 


train error 63.22509762903835
test error 70.87985056472863 


train error 66.72252882139763
test error 62.681397740181744 


train error 63.92260817139541
test error 71.9899503119304 


train error 64.69160740357884
test error 68.1333271318704 




**линейная модель показывает себя плохо, попробуем нелинейные**

# SVR

In [137]:
svr = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("svr", SVR())
])

In [140]:
# numerical
kfold_test(numerical_set, y, svr)

train error 58.656487827831825
test error 50.577027223215495 


train error 54.603523571056016
test error 66.87495923589405 


train error 57.65492013239953
test error 52.92653665773538 


train error 56.32301503863423
test error 60.35189063783314 


train error 57.17523812358849
test error 56.13262946569442 




In [141]:
# categorical
kfold_test(encoded_categorical, cat_y, svr)

train error 61.93511344396127
test error 51.919313528545295 


train error 57.101079600150314
test error 71.02411421825349 


train error 61.12979009890458
test error 55.35101079774404 


train error 59.0701357202972
test error 63.511802192373764 


train error 60.33389530927496
test error 58.623416234483315 




In [152]:
text_svr = svr = Pipeline(steps=[
    ("svd", TruncatedSVD(n_components=100)),
    ("scaler", StandardScaler()),
    ("svr", SVR())
])

In [153]:
kfold_test_text(text_train, y, text_svr)

train error 67.50812790608768
test error 58.96173699722473 


train error 63.11090871675973
test error 76.84624938505232 


train error 67.07060639715498
test error 60.81283797485942 


train error 64.70410736588752
test error 69.82076050483946 


train error 66.10760122006693
test error 64.66784497947513 




# Random Forest Regression

In [187]:
rf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("rf", RandomForestRegressor(n_estimators=100, min_samples_split=5, n_jobs=-1, criterion="mae",
                                 min_samples_leaf=10))
])

In [188]:
kfold_test(numerical_set, y, rf)

train error 41.529564315352694
test error 40.74387230514096 


train error 38.521383817427385
test error 53.55597014925373 


train error 41.02042116182572
test error 42.781210613598674 


train error 40.20000207382829
test error 48.23169435215947 


train error 40.17351928660307
test error 46.18604651162791 




In [172]:
kfold_test(encoded_categorical, cat_y, rf)

train error 57.14477999169781
test error 47.3093864013267 


train error 52.100944375259445
test error 67.60364013267 


train error 55.89954564315352
test error 52.59615448504984 


train error 54.34098132780083
test error 58.73886212624584 


train error 55.51774273858921
test error 54.22428571428571 




In [202]:
# text set
text_rf = svr = Pipeline(steps=[
    ("svd", TruncatedSVD(n_components=10)),
    ("scaler", StandardScaler()),
    ("svr", RandomForestRegressor(n_estimators=30, min_samples_split=2, n_jobs=-1, criterion="mae",
                                 min_samples_leaf=2))
])

In [203]:
kfold_test_text(text_train, y, text_rf)

train error 43.353492392807745
test error 59.16694306246546 


train error 39.71132780082988
test error 74.78736871199558 


train error 43.17894882434302
test error 63.56716417910448 


train error 41.65884833402461
test error 71.71810631229236 


train error 43.40595188718374
test error 71.12447397563677 




# MLP Regressor

In [234]:
mlpr = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(hidden_layer_sizes=(3, 3), random_state=42, max_iter=10000, solver="adam", activation="relu"))
])

In [232]:
kfold_test(numerical_set, y, mlpr)

train error 53.12277583473612
test error 49.39737872360799 


train error 49.09278572505419
test error 59.254346737204095 


train error 50.99910088618802
test error 280.4607629011353 


train error 52.48052141364388
test error 53.803443721812336 


train error 51.209738718416254
test error 65.10034726463721 




In [235]:
kfold_test(encoded_categorical, cat_y, mlpr)

train error 67.37440811148757
test error 55.28472185168597 


train error 60.63887254721764
test error 73.73801222975246 


train error 63.864090629181334
test error 61.380508733312816 


train error 62.641874336296475
test error 65.16675598664753 


train error 64.04691505781199
test error 65.480339167699 




In [244]:
mlpr_text = Pipeline(steps=[
    ("svd", TruncatedSVD(n_components=100)),
    ("mlp", MLPRegressor(hidden_layer_sizes=(3,), random_state=42, max_iter=10000, solver="adam", activation="relu"))
])

In [245]:
kfold_test_text(text_train, y, mlpr_text)

/home/alexey/development/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(


train error 57.980011861514534
test error 52.28229696383806 


train error 62.3738339223207
test error 70.31311479048804 


train error 56.48087514221192
test error 56.23714522808524 




/home/alexey/development/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(


train error 52.9769176318285
test error 64.46608538977958 


train error 52.99710753173935
test error 60.23649613346984 




/home/alexey/development/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(


## Не удалось определить хорошую модель, совмещение датасетов разных типов тоже не дало хорошего результата. Возможно, нужно больше времени на подбор гиперпааметров или генерацию новых признаков

**функция для заполнения Submission файла**

In [277]:
def make_submission(file, prediction_column, model, out_file):
    test_set = pd.read_csv(file)
    # предсказания на числовых данных
    needed_columns = ['accommodates', 'bathrooms', 'bedrooms',
   'beds', 'guests_included', 'minimum_nights', 'maximum_nights',
   'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
   'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
   'calculated_host_listings_count', 'reviews_per_month', 'cleaning_fee', 'host_response_rate', 'host_acceptance_rate']
    num_set = test_set.copy()[needed_columns]
    num_set["cleaning_fee"] = price_transformer(num_set["cleaning_fee"])   
    num_set["host_response_rate"] = percents_transformer(num_set["host_response_rate"])
    num_set["host_acceptance_rate"] = percents_transformer(num_set["host_acceptance_rate"])
    numerical_fill(num_set, needed_columns)
    predictions = model.predict(num_set)
    pred_col = pd.DataFrame({prediction_column: predictions})
    out_df = pd.concat([test_set, pred_col], axis=1)
    out_df.to_csv(out_file, index=False)
    

In [257]:
# тренируем на численных данных
fin_reg = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("rf", RandomForestRegressor(n_estimators=100, min_samples_split=5, n_jobs=-1, criterion="mae",
                                 min_samples_leaf=10))
])

In [258]:
fin_reg.fit(numerical_set, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(criterion='mae', min_samples_leaf=10,
                                       min_samples_split=5, n_jobs=-1))])

In [278]:
make_submission("Submission.csv", "price_prediction", fin_reg, "sub.csv")